In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.svm import LinearSVC



In [2]:
df = pd.read_excel('wideform.xlsx')

df.head(5)

,code_module,code_presentation,id_student,age_band,num_of_prev_attempts,studied_credits,final_result,date_registration,date_unregistration,days_from_due_A1,...,subpage_pre-0,subpage_1-15,subpage_16-30,subpage_31-45,subpage_46-60,url_pre-0,url_1-15,url_16-30,url_31-45,url_46-60
0,AAA,2013J,11391,55<=,0,240,Pass,-159.0,NaN,-1.0,...,11.0,9.0,2.0,1.0,NaN,NaN,1.0,NaN,NaN,NaN
1,AAA,2013J,28400,35-55,0,60,Pass,-53.0,NaN,3.0,...,26.0,17.0,16.0,1.0,1.0,10.0,9.0,11.0,1.0,1.0
2,AAA,2013J,31604,35-55,0,60,Pass,-52.0,NaN,-2.0,...,19.0,19.0,17.0,17.0,3.0,3.0,10.0,8.0,12.0,3.0
3,AAA,2013J,32885,0-35,0,60,Pass,-176.0,NaN,7.0,...,17.0,3.0,8.0,NaN,1.0,3.0,NaN,3.0,NaN,NaN
4,AAA,2013J,38053,35-55,0,60,Pass,-110.0,NaN,0.0,...,15.0,6.0,11.0,3.0,3.0,1.0,4.0,6.0,2.0,3.0


In [3]:
index_cols = ['code_module', 'code_presentation', 'id_student']
label = ['final_result']
# feature_cols is everything else



# I'm removing age_band for now, though so everything's a number.
# I'm also removing date_unregistration and days from due because
# those don't make sense to change NaNs to 0, so that'll take more thought
not_features = ['code_module', 'code_presentation', 'id_student', 'final_result', 'age_band', 'date_unregistration']
features = [col for col in df.columns if col not in not_features]

# Setting NaNs as 0, see caveats above
df.fillna(0, inplace=True)

# Set fail as 1, pass as 0
# Distinction currently counted as pass
# Withdrawl counted as fail
df.loc[df['final_result'] == "Fail", 'final_result'] = 1 
df.loc[df['final_result'] == "Withdrawn", 'final_result'] = 1 
df.loc[df['final_result'] == "Pass", 'final_result'] = 0
df.loc[df['final_result'] == "Distinction", 'final_result'] = 0

In [4]:
X_train, X_test, y_train, y_test= train_test_split(df[features],df[label], test_size=0.20)

In [5]:
pipe = Pipeline([('scaler', StandardScaler())])
pipe.fit(X_train, y_train)
X_train = pipe.transform(X_train)
X_test = pipe.transform(X_test)


In [6]:
X_train

array([[-0.34606865, -0.55760369, -0.2189122 , ..., -0.18200086,
        -0.41713002, -0.13537001],
       [-0.34606865, -0.55760369,  0.16111501, ...,  0.47721197,
         0.05486797, -0.34636286],
       [-0.34606865,  1.074374  , -0.07112384, ..., -0.51160727,
        -0.41713002, -0.13537001],
       ...,
       [-0.34606865,  1.074374  ,  0.92116944, ..., -0.51160727,
        -0.41713002, -0.34636286],
       [-0.34606865, -0.55760369,  0.96339468, ...,  2.12524404,
         3.35885391,  0.91959421],
       [-0.34606865, -0.55760369, -1.71790843, ...,  0.64201518,
        -0.18113103,  0.07562283]])

In [7]:
y_train = y_train.astype('int')
y_test = y_test.astype('int')

In [44]:
anova_filter = SelectKBest(f_classif, k=30) #Adjust K to see choice cutoffs
clf = LinearSVC(max_iter=1000000) #Trying to see if this will converge...
anova_svm = make_pipeline(anova_filter, clf)
anova_svm.fit(X_train, y_train['final_result'])

Pipeline(steps=[('selectkbest', SelectKBest(k=30)),
                ('linearsvc', LinearSVC(max_iter=1000000))])

In [45]:
boolean_mask = anova_svm[0].get_support()

In [46]:
chosen_features = [b for a, b in zip(boolean_mask, features) if a]

In [47]:
chosen_features

['num_of_prev_attempts',
 'days_from_due_A1',
 'score_A1',
 'score_A2',
 'forumng_pre-0',
 'forumng_1-15',
 'forumng_16-30',
 'forumng_31-45',
 'forumng_46-60',
 'homepage_pre-0',
 'homepage_1-15',
 'homepage_16-30',
 'homepage_31-45',
 'homepage_46-60',
 'oucontent_pre-0',
 'oucontent_1-15',
 'oucontent_16-30',
 'oucontent_31-45',
 'oucontent_46-60',
 'questionnaire_16-30',
 'questionnaire_31-45',
 'quiz_1-15',
 'quiz_31-45',
 'quiz_46-60',
 'subpage_pre-0',
 'subpage_1-15',
 'subpage_46-60',
 'url_1-15',
 'url_31-45',
 'url_46-60']

In [48]:
score = anova_svm.score(X_test, y_test)

score

0.7584830339321357

When k=10, the classifier chooses scores, forum, homepage, oucontent, subpage.
Score = 0.7564870259481038

When k=20, the classifier adds previous attempts, days from due A1, and url. Score = 0.7562652472832113

When k=30, the classifier adds questionnaire and quiz.
Score = 0.7584830339321357


For the LinearSVC with default parameters except for increasing iterations... it seems to converge at 1000000 iterations. Potentially, it would be smarter to adjust other parameters. A different classifier may also fit our data better. It would be nice to get a score over 80%.

In [49]:
from sklearn.tree import DecisionTreeClassifier

anova_filter = SelectKBest(f_classif, k=10) #Adjust K to see choice cutoffs
clf = DecisionTreeClassifier()
anova_svm = make_pipeline(anova_filter, clf)
anova_svm.fit(X_train, y_train['final_result'])

score = anova_svm.score(X_test, y_test)

score

0.6733200266134398

In [50]:
from sklearn.ensemble import RandomForestClassifier

anova_filter = SelectKBest(f_classif, k=10) #Adjust K to see choice cutoffs
clf = RandomForestClassifier()
anova_svm = make_pipeline(anova_filter, clf)
anova_svm.fit(X_train, y_train['final_result'])

score = anova_svm.score(X_test, y_test)

score

0.7604790419161677

In [51]:
from sklearn.naive_bayes import GaussianNB

anova_filter = SelectKBest(f_classif, k=10) #Adjust K to see choice cutoffs
clf = GaussianNB()
anova_svm = make_pipeline(anova_filter, clf)
anova_svm.fit(X_train, y_train['final_result'])

score = anova_svm.score(X_test, y_test)

score

0.5719671767575959

In [52]:
from sklearn.svm import SVC

anova_filter = SelectKBest(f_classif, k=10) #Adjust K to see choice cutoffs
clf = SVC()
anova_svm = make_pipeline(anova_filter, clf)
anova_svm.fit(X_train, y_train['final_result'])

score = anova_svm.score(X_test, y_test)

score

0.7584830339321357